In [7]:
%pip install fastapi pydantic toml openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add round summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/love_cruise.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round =0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

context: You have arrived at the port where you are to board a cruise to attend your best friend Alex’s wedding in Svalbard, Norway. Originally you were supposed to take a flight along with everyone else, but due to the limited seat capacity, Alex booked you a cruise instead. The cruise staff checked you in and directed you towards the ship, and urged you to hurry as it is almost time to leave...
actions:
1. Run towards the ship at full speed
2. Take the club car
3. Take your time because why not


I select option 3. TURN: 1 


context: As you decide to take your time, enjoying the cool sea breeze and the bustling atmosphere of the port, a vibrant figure catches your eye. Sophia, with her calico dress fluttering in the wind and a floppy hat perched jauntily on her head, grabs you by your forearm with a laugh. "You're not planning on missing this ship, are you?" she teases, her big brown eyes sparkling with adventure. Without waiting for your response, she starts dragging you towards the ship, her energy infectious.
actions:
1. Thank her and introduce yourself.
2. Ask her why she's in such a hurry.
3. Try to match her pace and energy.

I select option 3. TURN: 2 


context: Matching Sophia's pace, you find yourself swept up in her energetic stride, the two of you weaving through the crowd with ease. "This is the way to start an adventure!" she exclaims, her voice full of excitement. As you near the ship, her grip on your arm relaxes, but she doesn't let go. "I've always believed in making every moment count," she says, turning to you with a cheeky grin. "You seem like you could use a bit of adventure in your life too."
actions:
1. Agree with her and share your excitement.
2. Joke about how you're usually more of a planner.
3. Express surprise at her intuition.

I select option 3. TURN: 3 


context: Sophia's laughter rings out, clear and melodious, as you express your surprise. "It's not hard to spot," she says, still grinning. "You have that look of someone who's on the brink of something new and thrilling. Plus, your suit bag says 'I'm here for something important, but I'm not quite sure what yet.'" She winks, her energy undiminished even as you both finally step onto the ship. "Well, you're in good company. I'm here to explore, to find stories, and to live freely. I'm Sophia, by the way."
summary: Jack decides to take his time and ends up being spirited away by Sophia, a vibrant and adventurous soul, towards the ship. Their interaction is filled with energy, laughter, and the promise of adventure. Jack is intrigued by Sophia's intuition and her free-spirited nature as they board the ship together.

context: You make your way towards your suite, and you are immediately greeted by a spacious room with a balcony. As you set your luggage and items in place, the Captain, through the intercom, invites everyone to the top deck for food and drinks as the cruise departs. You...
actions:
1. Attend the party on the deck
2. Stay in your suite and turn on the TV
3. Go to the balcony and relax


I select option 1. TURN: 1 


context: Eager to immerse yourself in the lively atmosphere of the ship, you head to the deck. The air is filled with the sound of music, laughter, and the clinking of glasses. As you navigate through the crowd, you spot Sophia, her energy unmistakable even from a distance. She waves you over, a wide smile on her face. "Jack! Come meet some friends!" she exclaims, pulling you into a group that includes a girl with a light, carefree laugh, who introduces herself as Saoirse.
actions:
1. Chat more with Saoirse and learn about her.
2. Ask Sophia about the ship's destinations.
3. Suggest a dance with Sophia and Saoirse.

I select option 1. TURN: 2 


context: You turn your attention to Saoirse, intrigued by her easygoing vibe. "So, what brings you to this cruise?" you ask, leaning in to hear her over the music. Saoirse's eyes light up, "Ah, just the love for adventure and a good party," she says with a laugh, her Irish accent adding a melodic tone to her words. "What about you?" she asks, genuinely interested. As you share your own reasons, you find the conversation flowing effortlessly, her stories of travels and escapades captivating you.
actions:
1. Share a funny travel mishap of your own.
2. Ask Saoirse to recommend a drink.
3. Invite Saoirse to join you and Sophia for an adventure on the ship.

I select option 2. TURN: 3 


context: "You've got to try the ship's signature cocktail," Saoirse suggests with an enthusiastic nod. "It's called the Ocean's Whisper. Perfect blend of mystery and flavor, much like the sea itself." She waves over a waiter, ordering two Ocean's Whispers with a confident smile. As you wait, she regales you with a story about the most unforgettable drink she's ever had in a small pub in Ireland, making you laugh with her vivid descriptions and cheeky comments. The drinks arrive, and you toast to new friendships and adventures.
actions:
1. Suggest exploring the ship after the drinks.
2. Ask Saoirse about her favorite adventure.
3. Compliment Saoirse on her storytelling ability.

I select option 2. TURN: 4 


context: Taking a sip of the Ocean's Whisper, you lean in, intrigued. "What's been your favorite adventure so far?" you ask. Saoirse ponders for a moment, her gaze drifting off to the sea before she smiles, eyes twinkling. "It has to be the time I went cliff diving in Moher. The thrill, the sheer drop, the cold splash of the Atlantic... It was terrifying and exhilarating all at once!" Her enthusiasm is infectious, and you find yourself imagining the rush of such an adventure, the wildness of the Irish coastline vivid in your mind through her words.
actions:
1. Share your own adventurous experience.
2. Propose a toast to daring adventures.
3. Ask Saoirse for tips on overcoming fears.

I select option 2. TURN: 5 


context: Raising your glass high, you catch Saoirse's eye, saying, "To daring adventures and the courage to face them head-on." She grins, clinking her glass against yours. "To adventures and the friends we meet along the way," she adds, and you both take a hearty sip. The atmosphere around you is electric, filled with the promise of new experiences and the joy of making connections. As the ship's horn sounds, marking the beginning of your journey, you feel a sense of excitement for what lies ahead, knowing that this trip will be unforgettable.
summary: Jack attends the party on the deck, meeting Sophia and Saoirse. He finds a kindred spirit in Saoirse, bonding over shared interests in adventure and travel. Their conversation is lively, filled with stories and laughter, culminating in a toast to daring adventures as the ship sets sail.